In [ ]:
!pip install transformers datasets torch

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# GPT-2 often needs a padding token defined
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
# Apply this function to your dataset
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',        # Directory for saving results and checkpoints
    num_train_epochs=3,            # Number of training epochs
    per_device_train_batch_size=2, # Batch size per GPU (adjust based on memory)
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_here, # Replace with your training dataset
    eval_dataset=eval_dataset_here,   # Replace with your evaluation dataset
    data_collator=data_collator,
)

trainer.train()

In [ ]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

In [ ]:
from transformers import pipeline
text_generator = pipeline('text-generation', model='./fine_tuned_gpt2', tokenizer=tokenizer)

# Example generation
# result = text_generator("Your custom prompt here", max_length=100, num_return_sequences=1)